# Create toy population dataset

In [23]:
import numpy as np
import numba as nb
from numba import njit, prange

In [2]:
pop_size = 12
str_len = 10
half_len = str_len//2

In [3]:
pop = np.array([np.random.permutation(np.arange(0, str_len)) for n in np.arange(pop_size)])
pop

array([[8, 9, 0, 5, 3, 1, 7, 4, 2, 6],
       [2, 4, 7, 5, 6, 3, 8, 1, 9, 0],
       [5, 6, 7, 0, 4, 2, 1, 8, 3, 9],
       [4, 0, 7, 9, 5, 6, 1, 2, 3, 8],
       [8, 5, 7, 3, 0, 2, 9, 1, 6, 4],
       [0, 3, 4, 1, 2, 5, 8, 6, 7, 9],
       [8, 9, 0, 1, 2, 7, 5, 6, 3, 4],
       [2, 6, 7, 9, 4, 0, 8, 3, 5, 1],
       [5, 0, 4, 7, 3, 1, 9, 2, 6, 8],
       [9, 7, 6, 8, 2, 4, 3, 5, 1, 0],
       [4, 9, 5, 1, 8, 0, 6, 7, 2, 3],
       [2, 3, 6, 1, 0, 4, 8, 5, 9, 7]])

# Sequence Selection on a single parent array

In [4]:
def vecArange(starts, delta):
    return np.repeat(np.expand_dims(starts, 1), delta, axis=1)+np.arange(0, delta)

In [5]:
seq_start = np.random.randint(0,half_len,pop_size)
seq_start

array([0, 0, 4, 1, 3, 4, 4, 4, 0, 0, 0, 0])

In [6]:
pop

array([[8, 9, 0, 5, 3, 1, 7, 4, 2, 6],
       [2, 4, 7, 5, 6, 3, 8, 1, 9, 0],
       [5, 6, 7, 0, 4, 2, 1, 8, 3, 9],
       [4, 0, 7, 9, 5, 6, 1, 2, 3, 8],
       [8, 5, 7, 3, 0, 2, 9, 1, 6, 4],
       [0, 3, 4, 1, 2, 5, 8, 6, 7, 9],
       [8, 9, 0, 1, 2, 7, 5, 6, 3, 4],
       [2, 6, 7, 9, 4, 0, 8, 3, 5, 1],
       [5, 0, 4, 7, 3, 1, 9, 2, 6, 8],
       [9, 7, 6, 8, 2, 4, 3, 5, 1, 0],
       [4, 9, 5, 1, 8, 0, 6, 7, 2, 3],
       [2, 3, 6, 1, 0, 4, 8, 5, 9, 7]])

In [7]:
seq = vecArange(seq_start,half_len)
seq

array([[0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4],
       [4, 5, 6, 7, 8],
       [1, 2, 3, 4, 5],
       [3, 4, 5, 6, 7],
       [4, 5, 6, 7, 8],
       [4, 5, 6, 7, 8],
       [4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4]])

In [8]:
subseq = pop[np.arange(pop.shape[0])[:, None], seq]
subseq

array([[8, 9, 0, 5, 3],
       [2, 4, 7, 5, 6],
       [4, 2, 1, 8, 3],
       [0, 7, 9, 5, 6],
       [3, 0, 2, 9, 1],
       [2, 5, 8, 6, 7],
       [2, 7, 5, 6, 3],
       [4, 0, 8, 3, 5],
       [5, 0, 4, 7, 3],
       [9, 7, 6, 8, 2],
       [4, 9, 5, 1, 8],
       [2, 3, 6, 1, 0]])

## Remove subsequences from other parents

In [9]:
pop_roll = np.roll(pop,1,axis=0)
pop_roll

array([[2, 3, 6, 1, 0, 4, 8, 5, 9, 7],
       [8, 9, 0, 5, 3, 1, 7, 4, 2, 6],
       [2, 4, 7, 5, 6, 3, 8, 1, 9, 0],
       [5, 6, 7, 0, 4, 2, 1, 8, 3, 9],
       [4, 0, 7, 9, 5, 6, 1, 2, 3, 8],
       [8, 5, 7, 3, 0, 2, 9, 1, 6, 4],
       [0, 3, 4, 1, 2, 5, 8, 6, 7, 9],
       [8, 9, 0, 1, 2, 7, 5, 6, 3, 4],
       [2, 6, 7, 9, 4, 0, 8, 3, 5, 1],
       [5, 0, 4, 7, 3, 1, 9, 2, 6, 8],
       [9, 7, 6, 8, 2, 4, 3, 5, 1, 0],
       [4, 9, 5, 1, 8, 0, 6, 7, 2, 3]])

In [10]:
children = np.empty((pop.shape[0],str_len),int)
for n, seq in enumerate(subseq):
    child = pop_roll[n][np.isin(pop_roll[n],seq,invert=True)]
    child = np.insert(seq,seq_start[n],child)
    children[n] = child
print(pop[1])
# print(pop_roll[1])
print(children[1])

[2 4 7 5 6 3 8 1 9 0]
[8 9 0 3 1 2 4 7 5 6]


# Sequence Selection with combined parent arrays

In [11]:
parents = np.concatenate((pop,pop_roll),axis=1).reshape(-1,2,str_len)
parents

array([[[8, 9, 0, 5, 3, 1, 7, 4, 2, 6],
        [2, 3, 6, 1, 0, 4, 8, 5, 9, 7]],

       [[2, 4, 7, 5, 6, 3, 8, 1, 9, 0],
        [8, 9, 0, 5, 3, 1, 7, 4, 2, 6]],

       [[5, 6, 7, 0, 4, 2, 1, 8, 3, 9],
        [2, 4, 7, 5, 6, 3, 8, 1, 9, 0]],

       [[4, 0, 7, 9, 5, 6, 1, 2, 3, 8],
        [5, 6, 7, 0, 4, 2, 1, 8, 3, 9]],

       [[8, 5, 7, 3, 0, 2, 9, 1, 6, 4],
        [4, 0, 7, 9, 5, 6, 1, 2, 3, 8]],

       [[0, 3, 4, 1, 2, 5, 8, 6, 7, 9],
        [8, 5, 7, 3, 0, 2, 9, 1, 6, 4]],

       [[8, 9, 0, 1, 2, 7, 5, 6, 3, 4],
        [0, 3, 4, 1, 2, 5, 8, 6, 7, 9]],

       [[2, 6, 7, 9, 4, 0, 8, 3, 5, 1],
        [8, 9, 0, 1, 2, 7, 5, 6, 3, 4]],

       [[5, 0, 4, 7, 3, 1, 9, 2, 6, 8],
        [2, 6, 7, 9, 4, 0, 8, 3, 5, 1]],

       [[9, 7, 6, 8, 2, 4, 3, 5, 1, 0],
        [5, 0, 4, 7, 3, 1, 9, 2, 6, 8]],

       [[4, 9, 5, 1, 8, 0, 6, 7, 2, 3],
        [9, 7, 6, 8, 2, 4, 3, 5, 1, 0]],

       [[2, 3, 6, 1, 0, 4, 8, 5, 9, 7],
        [4, 9, 5, 1, 8, 0, 6, 7, 2, 3]]])

In [12]:
parents

array([[[8, 9, 0, 5, 3, 1, 7, 4, 2, 6],
        [2, 3, 6, 1, 0, 4, 8, 5, 9, 7]],

       [[2, 4, 7, 5, 6, 3, 8, 1, 9, 0],
        [8, 9, 0, 5, 3, 1, 7, 4, 2, 6]],

       [[5, 6, 7, 0, 4, 2, 1, 8, 3, 9],
        [2, 4, 7, 5, 6, 3, 8, 1, 9, 0]],

       [[4, 0, 7, 9, 5, 6, 1, 2, 3, 8],
        [5, 6, 7, 0, 4, 2, 1, 8, 3, 9]],

       [[8, 5, 7, 3, 0, 2, 9, 1, 6, 4],
        [4, 0, 7, 9, 5, 6, 1, 2, 3, 8]],

       [[0, 3, 4, 1, 2, 5, 8, 6, 7, 9],
        [8, 5, 7, 3, 0, 2, 9, 1, 6, 4]],

       [[8, 9, 0, 1, 2, 7, 5, 6, 3, 4],
        [0, 3, 4, 1, 2, 5, 8, 6, 7, 9]],

       [[2, 6, 7, 9, 4, 0, 8, 3, 5, 1],
        [8, 9, 0, 1, 2, 7, 5, 6, 3, 4]],

       [[5, 0, 4, 7, 3, 1, 9, 2, 6, 8],
        [2, 6, 7, 9, 4, 0, 8, 3, 5, 1]],

       [[9, 7, 6, 8, 2, 4, 3, 5, 1, 0],
        [5, 0, 4, 7, 3, 1, 9, 2, 6, 8]],

       [[4, 9, 5, 1, 8, 0, 6, 7, 2, 3],
        [9, 7, 6, 8, 2, 4, 3, 5, 1, 0]],

       [[2, 3, 6, 1, 0, 4, 8, 5, 9, 7],
        [4, 9, 5, 1, 8, 0, 6, 7, 2, 3]]])

In [13]:
seq_start = np.random.randint(0,half_len,pop_size).repeat(2) #repeat is so pairs of parents have same subseq
seq_start

array([2, 2, 2, 2, 2, 2, 3, 3, 2, 2, 1, 1, 2, 2, 4, 4, 0, 0, 0, 0, 1, 1,
       4, 4])

In [14]:
parent_seq = vecArange(seq_start,half_len).reshape(-1,2,half_len)
parent_seq

array([[[2, 3, 4, 5, 6],
        [2, 3, 4, 5, 6]],

       [[2, 3, 4, 5, 6],
        [2, 3, 4, 5, 6]],

       [[2, 3, 4, 5, 6],
        [2, 3, 4, 5, 6]],

       [[3, 4, 5, 6, 7],
        [3, 4, 5, 6, 7]],

       [[2, 3, 4, 5, 6],
        [2, 3, 4, 5, 6]],

       [[1, 2, 3, 4, 5],
        [1, 2, 3, 4, 5]],

       [[2, 3, 4, 5, 6],
        [2, 3, 4, 5, 6]],

       [[4, 5, 6, 7, 8],
        [4, 5, 6, 7, 8]],

       [[0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4]],

       [[0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4]],

       [[1, 2, 3, 4, 5],
        [1, 2, 3, 4, 5]],

       [[4, 5, 6, 7, 8],
        [4, 5, 6, 7, 8]]])

In [35]:
p1 = parents[0][0]
p2 = parents[0][1]
print(p1,p2,sep="\n")

[8 9 0 5 3 1 7 4 2 6]
[2 3 6 1 0 4 8 5 9 7]


In [24]:
@njit(parallel=True)
def in1d(array, remove):
    out = np.empty(array.shape[0], dtype=np.bool_)
    remove = set(remove)
    for i in prange(array.shape[0]):
        if array[i] in remove:
            out[i] = False
        else:
            out[i] = True
    return out

In [95]:
def roll(arr, num, fill_value=-1):
    result = np.empty_like(arr)
    if num > 0:
        result[:num] = fill_value
        result[num:] = arr[:-num]
    elif num < 0:
        result[num:] = fill_value
        result[:num] = arr[-num:]
    else:
        result = arr
    return result

In [99]:
@njit(parallel=False, fastmath=True)
def order_xover_2par(p1, p2, slice):
    inv_slice = in1d(np.arange(p1.size),slice)
    child = np.zeros_like(p1)
    child[slice] = p1[slice]
    p2_diff = in1d(p2,child[slice])
    child[inv_slice] = p2[p2_diff]
    return child

In [40]:
start = np.random.randint(1, p1.size // 2)
slice = np.arange(start, start + p1.size // 2)
slice

array([3, 4, 5, 6, 7])

In [98]:
print(print(p1,p2,sep="\n"))
order_xover_2par(p1,p2,slice)

[8 9 0 5 3 1 7 4 2 6]
[2 3 6 1 0 4 8 5 9 7]
None
child from p1:  [0 0 0 5 3 1 7 4 0 0]
p2 with mask:  [2 6 0 8 9]
child:  [2 6 0 5 3 1 7 4 8 9]


array([2, 6, 0, 5, 3, 1, 7, 4, 8, 9])

In [100]:
pop

array([[8, 9, 0, 5, 3, 1, 7, 4, 2, 6],
       [2, 4, 7, 5, 6, 3, 8, 1, 9, 0],
       [5, 6, 7, 0, 4, 2, 1, 8, 3, 9],
       [4, 0, 7, 9, 5, 6, 1, 2, 3, 8],
       [8, 5, 7, 3, 0, 2, 9, 1, 6, 4],
       [0, 3, 4, 1, 2, 5, 8, 6, 7, 9],
       [8, 9, 0, 1, 2, 7, 5, 6, 3, 4],
       [2, 6, 7, 9, 4, 0, 8, 3, 5, 1],
       [5, 0, 4, 7, 3, 1, 9, 2, 6, 8],
       [9, 7, 6, 8, 2, 4, 3, 5, 1, 0],
       [4, 9, 5, 1, 8, 0, 6, 7, 2, 3],
       [2, 3, 6, 1, 0, 4, 8, 5, 9, 7]])

In [105]:
pop_roll = np.empty_like(pop)
pop_roll.fill(-1)
pop_roll[1:] = pop[0:-1]
pop_roll[0] = pop[-1]
pop_roll

array([[2, 3, 6, 1, 0, 4, 8, 5, 9, 7],
       [8, 9, 0, 5, 3, 1, 7, 4, 2, 6],
       [2, 4, 7, 5, 6, 3, 8, 1, 9, 0],
       [5, 6, 7, 0, 4, 2, 1, 8, 3, 9],
       [4, 0, 7, 9, 5, 6, 1, 2, 3, 8],
       [8, 5, 7, 3, 0, 2, 9, 1, 6, 4],
       [0, 3, 4, 1, 2, 5, 8, 6, 7, 9],
       [8, 9, 0, 1, 2, 7, 5, 6, 3, 4],
       [2, 6, 7, 9, 4, 0, 8, 3, 5, 1],
       [5, 0, 4, 7, 3, 1, 9, 2, 6, 8],
       [9, 7, 6, 8, 2, 4, 3, 5, 1, 0],
       [4, 9, 5, 1, 8, 0, 6, 7, 2, 3]])